In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('train.csv')

In [18]:
df.head(20)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,False,False,False,False,True,False,False,False,True,False
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,False,False,False,False,True,False,False,False,True,False
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,False,False,False,False,True,False,False,False,True,False
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,False,False,False,False,True,False,False,False,False,False
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,False,False,False,False,True,False,False,False,True,False
5,6,50,85.0,14115,5,5,1993,1995,0.0,732,...,False,False,False,False,True,False,False,False,True,False
6,7,20,75.0,10084,8,5,2004,2005,186.0,1369,...,False,False,False,False,True,False,False,False,True,False
7,8,60,69.0,10382,7,6,1973,1973,240.0,859,...,False,False,False,False,True,False,False,False,True,False
8,9,50,51.0,6120,7,5,1931,1950,0.0,0,...,False,False,False,False,True,False,False,False,False,False
9,10,190,50.0,7420,5,6,1939,1950,0.0,851,...,False,False,False,False,True,False,False,False,True,False


In [3]:
# 1. Điền giá trị thiếu (NaN) với giá trị mặc định hoặc giá trị hợp lý
df.fillna({
    'LotFrontage': df['LotFrontage'].median(),  # Điền giá trị trung vị cho LotFrontage
    'Alley': 'NoAlley',  # Giả sử không có Alley là 'NoAlley'
    'PoolQC': 'NoPool',  # Giả sử không có Pool là 'NoPool'
    'Fence': 'NoFence',  # Giả sử không có Fence là 'NoFence'
    'MiscFeature': 'NoFeature'  # Giả sử không có MiscFeature là 'NoFeature'
}, inplace=True)

In [4]:

# 2. Chuyển đổi các cột phân loại (categorical) thành kiểu category để tiết kiệm bộ nhớ
categorical_columns = ["MSZoning", "Street", "LotShape", "LandContour", "Utilities", "SaleType", "SaleCondition", "Alley", "PoolQC", "Fence", "MiscFeature"]
for col in categorical_columns:
    df[col] = df[col].astype("category")

In [5]:
# Chuyển đổi các cột phân loại thành biến giả (dummy variables)
df = pd.get_dummies(df, drop_first=True)

In [6]:
# 3. Xử lý giá trị thiếu bằng SimpleImputer
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [7]:
X = df_imputed.drop(columns=['SalePrice'])
y = df_imputed['SalePrice']

In [8]:
# Khởi tạo mô hình RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor()

In [9]:
# Lấy ra tầm quan trọng của các biến (feature importances)
importances = model.feature_importances_
features = X.columns

In [11]:
# Tạo dataframe tầm quan trọng của các biến
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)


In [12]:
important_features = feature_importances[feature_importances['Importance'] > 0.01]['Feature']


In [13]:
important_features = feature_importances[feature_importances['Importance'] > 0.01]['Feature']


In [14]:
X_important = X[important_features]


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_important, y, test_size=0.2, random_state=42)


In [16]:
model_important = RandomForestRegressor()
model_important.fit(X_train, y_train)


RandomForestRegressor()

In [17]:

# Dự đoán và đánh giá mô hình
y_pred = model_important.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 786654343.2627351
